SCRIPT DESCARGA DE CAUDALES EN AUTOMATICO

In [14]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

# Configurar el driver de Chrome
def configure_driver(download_path):
    options = webdriver.ChromeOptions()
    prefs = {
        "download.default_directory": download_path,
        "profile.default_content_settings.popups": 0,
        "download.prompt_for_download": False,
        "directory_upgrade": True,
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    return driver

# Ruta base de descarga
base_download_path = r"C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM"  # Reemplaza con tu ruta de descarga

# Configurar el driver para las descargas
driver = configure_driver(base_download_path)

try:
    driver.get("https://www.sqmsenlinea.com/agua-industrial/118")
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "select#acp_id"))
    )

    # Seleccionar el dropdown dejandolo disponible
    select_acp = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#acp_id")))
    select = Select(select_acp)
    options = select.options
    # Iterar sobre las opciones del dropdown
    
    for element in range(1, len(select.options)):
        select.select_by_index(element)
        time.sleep(5)
        # Seleccionar la descarga de archivo
        driver.find_element(By.CSS_SELECTOR, "a.btn.btn--white.download_button").click()
        
        time.sleep(5)
        
        # Vuelve a seleccionar el dropdown
        select_acp = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "select#acp_id")))
        select = Select(select_acp)
        options = select.options

     

except Exception as e:
    print(f"Ha ocurrido un error: {e}")
finally:
    print("Proceso completado")


Cerrando el navegador


PROCESAR LAS DESCARGAS DE CAUDALES

Temporalidad de caudales en la base de datos

In [5]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Credenciales db
host = 'localhost'
database = 'DB_SQM-SALAR'
username = 'postgres'  
password = '1234' 

# Crea la coneccion
conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
)

engine = create_engine('postgresql://'+username+':'+password+'@'+host+'/'+database)
table_name1 = 'SQM-Hidroquimica'
table_name2 = 'SQM-Infraestructuras'
table_name3 = 'SQM-Extraccion_Caudal_Mensual'


# df_hidroquimica = pd.read_sql_table(table_name1, con=engine)#Trae la tabla de la bda un dataframe
# df_infraestructuras = pd.read_sql_table(table_name2, con=engine)#
df_extraccion = pd.read_sql_table(table_name3, con=engine)#

JUNTAR TODOS LOS DATAFRAMES EN UNOS SOLO

In [26]:
import pandas as pd
import os

# Ruta a la carpeta que contiene los archivos Excel
ruta_carpeta = r"C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM"

# Crear una lista para almacenar los DataFrames
dataframes = []

# Recorrer todos los archivos en la carpeta
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.xlsx') or archivo.endswith('.xls'):  # Filtrar solo archivos Excel
        archivo_completo = os.path.join(ruta_carpeta, archivo)
        
        # Leer el archivo Excel
        df = pd.read_excel(archivo_completo)
        
        # Agregar el DataFrame a la lista
        dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_concatenado = pd.concat(dataframes, ignore_index=True)

print(df_concatenado)

                    Fecha Punto de monitoreo Unidad  Valor
0     2019-05-25 00:00:00           Allana 1    L/s   0.00
1     2019-05-26 00:00:00           Allana 1    L/s   0.00
2     2019-05-27 00:00:00           Allana 1    L/s   0.00
3     2019-05-28 00:00:00           Allana 1    L/s  15.05
4     2019-05-29 00:00:00           Allana 1    L/s  39.41
...                   ...                ...    ...    ...
8166  2024-06-12 00:00:00          Socaire 5    L/s  64.18
8167  2024-06-13 00:00:00          Socaire 5    L/s  64.13
8168  2024-06-14 00:00:00          Socaire 5    L/s  64.16
8169  2024-06-15 00:00:00          Socaire 5    L/s  63.95
8170  2024-06-16 00:00:00          Socaire 5    L/s  64.09

[8171 rows x 4 columns]


In [28]:
data1 = df_concatenado

In [29]:
# convertir Fecha a datetime
data1['Fecha'] = pd.to_datetime(data1['Fecha'])


In [31]:
df = data1
df['Fecha'] = pd.to_datetime(df['Fecha'])  # Asegurarse de que la columna de Fecha sea de tipo datetime

# Crear una columna que contenga el primer día del mes
df['Periodo'] = df['Fecha'].dt.to_period('M').dt.to_timestamp()

# Agrupar por punto de monitoreo y por el primer día del mes, luego calcular el promedio
promedio_mensual = df.groupby(['Punto de monitoreo', 'Periodo'])['Valor'].mean().reset_index()

# Renombrar la columna de Valor para reflejar que es un promedio
promedio_mensual.rename(columns={'Valor': 'promedio_mensual'}, inplace=True)
promedio_mensual

,Punto de monitoreo,Periodo,promedio_mensual
0,Allana 1,2019-05-01,19.050000
1,Allana 1,2019-06-01,35.610000
2,Allana 1,2019-07-01,36.065484
3,Allana 1,2019-08-01,34.586129
4,Allana 1,2019-09-01,38.394667
...,...,...,...
270,Socaire 5,2024-02-01,62.200000
271,Socaire 5,2024-03-01,64.064194
272,Socaire 5,2024-04-01,63.877333
273,Socaire 5,2024-05-01,63.905484


In [19]:
# Columna promedio mensual maximo 3 decimales
promedio_mensual['promedio_mensual'] = promedio_mensual['promedio_mensual']

In [33]:
promedio_mensual['dias_por_mes'] = promedio_mensual['Periodo'].dt.daysinmonth

Calculo de Volumen explotado M3/MES

In [34]:
# Columna Volúmen explotado (m3/mes)  es igual a (promedio_mensual * 86.4) * dias_por_mes
promedio_mensual['Volumen explotado (m3/mes)'] = (promedio_mensual['promedio_mensual'] * 86.4) * promedio_mensual['dias_por_mes']


In [38]:
# Añadir columna Fecha_Update
promedio_mensual['Fecha_Update'] = pd.to_datetime('today')


RENOMBRAR COLUMNAS

In [36]:
promedio_mensual.rename(columns={'Punto de monitoreo': 'Punto'}, inplace=True)
promedio_mensual.rename(columns={'promedio_mensual': 'Caudal medio equivalente (l/s)'}, inplace=True)

In [40]:
# Filtrar desde el 2023-11-01 en adelante
promedio_mensual_Update = promedio_mensual[promedio_mensual['Periodo'] >= '2023-11-01']
promedio_mensual_Update

,Punto,Periodo,Caudal medio equivalente (l/s),dias_por_mes,Volumen explotado (m3/mes),Fecha_Update
54,Allana 1,2023-11-01,0.345667,30,895.968,2024-06-25 11:54:07.922497
55,Allana 1,2023-12-01,0.000000,31,0.000,2024-06-25 11:54:07.922497
56,Allana 1,2024-01-01,0.000000,31,0.000,2024-06-25 11:54:07.922497
57,Allana 1,2024-02-01,0.018276,29,45.792,2024-06-25 11:54:07.922497
58,Allana 1,2024-03-01,3.088065,31,8271.072,2024-06-25 11:54:07.922497
59,Allana 1,2024-04-01,4.711667,30,12212.640,2024-06-25 11:54:07.922497
60,Allana 1,2024-05-01,4.671935,31,12513.312,2024-06-25 11:54:07.922497
61,Allana 1,2024-06-01,4.709375,30,12206.700,2024-06-25 11:54:07.922497
116,CA-2015,2023-11-01,32.815333,30,85057.344,2024-06-25 11:54:07.922497
117,CA-2015,2023-12-01,33.864516,31,90702.720,2024-06-25 11:54:07.922497


In [45]:
# Eliminar de este df columna dias_por_mes
# promedio_mensual_Update = promedio_mensual_Update.drop(['dias_por_mes'], axis=1)
promedio_mensual_Update.columns

Index(['Punto', 'Periodo', 'Caudal medio equivalente (l/s)',
       'Volumen explotado (m3/mes)', 'Fecha_Update'],
      dtype='object')

In [48]:
nuevo_orden_columnas = ['Punto', 'Periodo', 'Volumen explotado (m3/mes)',
       'Caudal medio equivalente (l/s)', 'Fecha_Update']

# Reindexar el DataFrame usando la lista
df_ordenado_update = promedio_mensual_Update[nuevo_orden_columnas]
df_ordenado_update.columns

Index(['Punto', 'Periodo', 'Volumen explotado (m3/mes)',
       'Caudal medio equivalente (l/s)', 'Fecha_Update'],
      dtype='object')

In [46]:
# eliminar columna Object_ID de dataframe df_extraccion
# df_extraccion.drop('Object_ID', axis=1, inplace=True)

df_extraccion.columns


Index(['Punto', 'Periodo', 'Volumen explotado (m3/mes)',
       'Caudal medio equivalente (l/s)', 'Fecha_Update'],
      dtype='object')

UNIR AMBOS DATAFRAME

In [49]:
# UNIR MEDIANTE CONCAT df_extraccion y df_ordenado_update
df_update = pd.concat([df_extraccion, df_ordenado_update], ignore_index=True)

In [51]:
df_update

,Punto,Periodo,Volumen explotado (m3/mes),Caudal medio equivalente (l/s),Fecha_Update
0,Allana-1,2008-07-31,0.000,0.000000,NaT
1,Allana-1,2008-08-31,0.000,0.000000,NaT
2,Allana-1,2008-09-30,16355.520,6.310000,NaT
3,Allana-1,2008-10-31,70575.840,26.350000,NaT
4,Allana-1,2008-11-30,91497.600,35.300000,NaT
...,...,...,...,...,...
1195,Socaire 5,2024-02-01,155848.320,62.200000,2024-06-25 11:54:07.922497
1196,Socaire 5,2024-03-01,171589.536,64.064194,2024-06-25 11:54:07.922497
1197,Socaire 5,2024-04-01,165570.048,63.877333,2024-06-25 11:54:07.922497
1198,Socaire 5,2024-05-01,171164.448,63.905484,2024-06-25 11:54:07.922497


In [52]:
df_update.to_sql(table_name3, con=engine, if_exists='replace', index=False)

200

### Detalle Proceso Actualizacion de BD de Extraccion de Aguas
- **Incremento de datos**: El número de datos aumentó de 1168 a 1200.

- **Formato de fechas**: Las fechas se han convertido al formato `datetime`.

- **Formato de valores**: Los valores están en formato `Float`.

- **Cálculo de Volumen Explotado (m³/mes)**:

  - **Fórmula**: ((L/s * 86.4)* días del mes)


Exportar BD a Excel en ruta especificada.

In [2]:


ruta_exportacion = r'G:\Unidades compartidas\03.1 Proyectos SQM\SQM-049_Informe Efectividad L4-12 (17Jul2024)\04.SIG-BD\06_Export_BD\Hidrogeología\BD-SQM-CAUDALES'
df_update.to_excel(ruta_exportacion, index=False)


NameError: name 'df_update' is not defined

In [6]:
df_extraccion

,Punto,Periodo,Volumen explotado (m3/mes),Caudal medio equivalente (l/s),Fecha_Update
0,Allana-1,2008-07-31,0.000,0.000000,NaT
1,Allana-1,2008-08-31,0.000,0.000000,NaT
2,Allana-1,2008-09-30,16355.520,6.310000,NaT
3,Allana-1,2008-10-31,70575.840,26.350000,NaT
4,Allana-1,2008-11-30,91497.600,35.300000,NaT
...,...,...,...,...,...
1195,Socaire 5,2024-02-01,155848.320,62.200000,2024-06-25 11:54:07.922497
1196,Socaire 5,2024-03-01,171589.536,64.064194,2024-06-25 11:54:07.922497
1197,Socaire 5,2024-04-01,165570.048,63.877333,2024-06-25 11:54:07.922497
1198,Socaire 5,2024-05-01,171164.448,63.905484,2024-06-25 11:54:07.922497


In [7]:
ruta_exportacion = r'G:\Unidades compartidas\03.1 Proyectos SQM\SQM-049_Informe Efectividad L4-12 (17Jul2024)\04.SIG-BD\06_Export_BD\Hidrogeología\BD-SQM-CAUDALES\CAUDALES_MENSUALES_SQM.xlsx'
df_extraccion.to_excel(ruta_exportacion, index=False)

POBLAR BASE DE DATOS DE PROYECTO PLANIFICADOR

##CONECCION A LA BD

In [24]:

import pandas as pd
from sqlalchemy import create_engine

# Parámetros de conexión
server = 'FRANCISCO\FLOWDATABASE'
database = 'FLOW- Planificador'  
username = 'sa' 
password = '1234' 
driver = 'ODBC Driver 17 for SQL Server'

# Cadena de conexión
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'

# Crear el engine de SQLAlchemy
engine = create_engine(connection_string)

# Nombre de la tabla que quieres extraer
table_name = 'Proyectos'

# Leer los datos de la tabla en un DataFrame de pandas
df = pd.read_sql_table(table_name, con=engine)

<>:5: SyntaxWarning: invalid escape sequence '\F'
<>:5: SyntaxWarning: invalid escape sequence '\F'
C:\Users\fdona\AppData\Local\Temp\ipykernel_11484\2134373896.py:5: SyntaxWarning: invalid escape sequence '\F'
  server = 'FRANCISCO\FLOWDATABASE'


In [25]:
df

,id_proyecto,Nombre_Proyecto,id_equipo,Descripcion,Budget
0,1,Variable cambio Climatico,3,Cliente AMS,1183200.00
1,2,Hidrogeología para LB y Estudios de EIA Línea ...,2,Hidrogeología para LB y Estudios de EIA Línea ...,2312000.00
2,3,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,1,Implementación de Modelo Balance de Agua,22193997.60
3,4,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,2,Gerenciamiento del proyecto,2739783.40
4,5,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,3,Implementación de Modelo Balance de Agua,11341858.91
5,6,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,2,Simulación de Escenarios,13480214.57
6,7,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,3,Gerenciamiento del proyecto,2629561.71
7,8,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,3,Implementación de Modelo Balance de Agua,3780619.64
8,9,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,2,Simulación de Escenarios,16475817.80
9,10,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,1,Análisis de Seguridad Hídrica,8613207.80


In [76]:
df.to_excel('Proyectos.xlsx', index=False)

In [20]:
df_update = pd.read_excel('Proyectos.xlsx')
df_update

,id_proyecto,Nombre_Proyecto,id_equipo,Descripcion,Budget
0,NaN,Variable cambio Climatico,3,Cliente AMS,1.183200e+06
1,NaN,Hidrogeología para LB y Estudios de EIA Línea ...,2,Hidrogeología para LB y Estudios de EIA Línea ...,2.312000e+06
2,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,1,Implementación de Modelo Balance de Agua,2.219400e+07
3,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,2,Gerenciamiento del proyecto,2.739783e+06
4,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,3,Implementación de Modelo Balance de Agua,1.134186e+07
5,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,2,Simulación de Escenarios,1.348021e+07
6,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,3,Gerenciamiento del proyecto,2.629562e+06
7,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,3,Implementación de Modelo Balance de Agua,3.780620e+06
8,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,2,Simulación de Escenarios,1.647582e+07
9,NaN,MODELO DE BALANCE DE AGUA ZONA NORTE AMSA,1,Análisis de Seguridad Hídrica,8.613208e+06


In [23]:
# eliminar columna project_id de dataframe df_update
df_update.drop('id_proyecto', axis=1, inplace=True)
df_update.to_sql(table_name, con=engine, if_exists='append', index=False)

25